In [1]:
import tkinter as tk
from tkinter import ttk, scrolledtext, font
from datetime import datetime
import json
import os
import serial
import time

# --- アプリケーションの設定と状態を管理する変数 ---
CONFIG_FILE = "motor_config.json"
SERIAL_PORT = 'COM3'  # ★★★ ご自身の環境に合わせて変更してください ★★★
BAUD_RATE = 9600
arduino = None

app_settings = {
    "jog_speed": 10.0,
    "goto_speed": 30.0,
    "default_seq_speed": 20.0
}
current_position = 0.0
saved_positions = {"P1": 0.0, "P2": 0.0, "P3": 0.0}
position_history = []
motor_is_moving = False
sequence_to_run = []
current_seq_step = 0
widgets_to_disable = []

# --- ファイル保存・読込機能 ---
def save_data():
    seq_data = []
    if 'seq_entries' in globals() and seq_entries:
        for i in range(5):
            seq_data.append({
                'dist': seq_entries[i]['dist_var'].get(),
                'speed': seq_entries[i]['speed_var'].get(),
                'wait': seq_entries[i]['wait_var'].get()
            })
    
    config_data = {
        "app_settings": app_settings,
        "saved_positions": saved_positions,
        "sequence": seq_data
    }
    try:
        with open(CONFIG_FILE, 'w') as f:
            json.dump(config_data, f, indent=4)
        print("設定を保存しました。")
    except Exception as e:
        print(f"設定の保存中にエラーが発生しました: {e}")

def load_data():
    global saved_positions, app_settings
    if os.path.exists(CONFIG_FILE):
        try:
            with open(CONFIG_FILE, 'r') as f:
                config_data = json.load(f)
                app_settings.update(config_data.get("app_settings", {}))
                saved_positions = config_data.get("saved_positions", saved_positions)
                
                jog_speed_var.set(app_settings["jog_speed"])
                goto_speed_var.set(app_settings["goto_speed"])
                seq_speed_var.set(app_settings["default_seq_speed"])

                seq_data = config_data.get("sequence", [])
                if 'seq_entries' in globals() and seq_entries:
                    for i, item in enumerate(seq_data):
                        if i < len(seq_entries):
                            seq_entries[i]['dist_var'].set(item.get('dist', ''))
                            speed_val = item.get('speed', '') or app_settings["default_seq_speed"]
                            seq_entries[i]['speed_var'].set(speed_val)
                            seq_entries[i]['wait_var'].set(item.get('wait', ''))
            log_message("保存された設定を読み込みました。")
        except Exception as e:
            log_message(f"設定の読み込み中にエラー: {e}", "red")

def on_closing():
    save_data()
    if arduino and arduino.is_open:
        arduino.close()
    window.destroy()

# --- 状態管理と通信 ---
def update_motor_status(is_moving):
    global motor_is_moving
    motor_is_moving = is_moving
    if is_moving:
        status_var.set("移動中"); status_label.config(foreground="white", background="#E74C3C")
        for widget in widgets_to_disable: widget.config(state=tk.DISABLED)
    else:
        status_var.set("待機中"); status_label.config(foreground="white", background="#2ECC71")
        for widget in widgets_to_disable: widget.config(state=tk.NORMAL)
        if sequence_to_run and current_seq_step < len(sequence_to_run):
            execute_next_sequence_step()

def connect_serial():
    global arduino
    try:
        arduino = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=0.1)
        time.sleep(2) # 接続確立のための待ち時間
        log_message(f"Arduinoに接続しました: {SERIAL_PORT}")
        update_motor_status(False)
    except serial.SerialException as e:
        log_message(f"Arduinoに接続できません: {e}", "red")
        update_motor_status(False)

def send_command(command):
    if arduino and arduino.is_open:
        log_message(f"送信: {command.strip()}", "blue")
        arduino.write(command.encode('utf-8'))
    else:
        log_message("エラー: Arduinoが接続されていません", "red")
        
def read_from_arduino():
    if arduino and arduino.is_open:
        try:
            while arduino.in_waiting > 0:
                line = arduino.readline().decode('utf-8', errors='ignore').strip()
                if not line: continue
                
                if line == "STATUS:MOVING":
                    update_motor_status(True)
                elif line == "STATUS:IDLE":
                    update_motor_status(False)
                else:
                    log_message(f"受信: {line}")
        except (serial.SerialException, TypeError) as e:
            log_message(f"Arduinoとの通信エラー: {e}", "red")
    
    window.after(100, read_from_arduino) # 100msごとにArduinoからの信号を確認

def _update_position(distance, move_type="移動"):
    global current_position; old_position = current_position; current_position += distance
    timestamp = datetime.now().strftime("%H:%M:%S"); history_entry = f"[{timestamp}] {move_type}: {old_position:.3f} -> {current_position:.3f} (Δ {distance:+.3f})"
    position_history.append(history_entry); update_displays()

def _send_move_command(distance, speed, move_type="移動"):
    if motor_is_moving: return
    command = f"o,{distance},{speed}\n"; send_command(command)
    _update_position(distance, move_type) # 送信と同時にGUI上の座標は更新

# --- GUIイベント関数 (ロジックはシミュレーション版と同じ) ---
def emergency_stop(): send_command("s\n"); log_message("!!!!! 緊急停止 !!!!!", "red"); update_motor_status(False)
def set_origin(): global current_position; _update_position(-current_position, "原点設定"); log_message("現在位置を原点に設定しました。")
def goto_absolute_position():
    try: target_pos = float(goto_pos_entry.get()); distance_to_move = target_pos - current_position; _send_move_command(distance_to_move, app_settings["goto_speed"], "絶対位置移動")
    except ValueError: log_message("エラー: 目標位置に数値を入力してください", "red")
def jog_forward(): _send_move_command(jog_step_var.get(), app_settings["jog_speed"], "ジョグ (+)")
def jog_backward(): _send_move_command(-jog_step_var.get(), app_settings["jog_speed"], "ジョグ (-)")
def handle_key_press(event):
    if motor_is_moving or event.widget.winfo_class() == 'TEntry': return
    if event.keysym == "Right": jog_forward()
    elif event.keysym == "Left": jog_backward()
def save_position(p_name): saved_positions[p_name] = current_position; update_displays(); log_message(f"位置 {current_position:.3f} mm を{p_name}に保存しました。")
def goto_position(p_name): target_pos = saved_positions[p_name]; distance_to_move = target_pos - current_position; _send_move_command(distance_to_move, app_settings["goto_speed"], f"{p_name}へ移動")
def reset_position(p_name): saved_positions[p_name] = 0.0; update_displays(); log_message(f"ポジション {p_name} をリセットしました。")
def run_sequence():
    global sequence_to_run, current_seq_step
    if motor_is_moving: return
    sequence_to_run = []; current_seq_step = 0
    for i in range(5):
        dist_str = seq_entries[i]['dist_var'].get(); speed_str = seq_entries[i]['speed_var'].get(); wait_str = seq_entries[i]['wait_var'].get()
        if dist_str and speed_str and wait_str:
            try: sequence_to_run.append({'dist': float(dist_str), 'speed': float(speed_str), 'wait': float(wait_str)})
            except ValueError: log_message(f"シーケンス {i+1}行目に不正な数値があります。", "red"); sequence_to_run = []; return
    if not sequence_to_run: log_message("シーケンスが入力されていません。", "orange"); return
    log_message("シーケンス実行開始..."); execute_next_sequence_step()
def execute_next_sequence_step():
    global current_seq_step
    if not sequence_to_run or current_seq_step >= len(sequence_to_run):
        if sequence_to_run: log_message("シーケンス完了。")
        sequence_to_run.clear(); current_seq_step = 0
        return
    step = sequence_to_run[current_seq_step]
    log_message(f"  -> ステップ {current_seq_step + 1}/{len(sequence_to_run)} を実行")
    current_seq_step += 1
    wait_time_ms = int(step['wait'] * 1000)
    window.after(wait_time_ms, lambda: _send_move_command(step['dist'], step['speed'], f"シーケンス {current_seq_step}"))
def clear_sequence():
    for i in range(5): 
        seq_entries[i]['dist_var'].set(""); seq_entries[i]['speed_var'].set(app_settings["default_seq_speed"]); seq_entries[i]['wait_var'].set("")
def log_message(message, color="black"):
    log_area.config(state=tk.NORMAL); log_area.insert(tk.END, f"[{datetime.now().strftime('%H:%M:%S')}] {message}\n")
    log_area.see(tk.END); log_area.config(state=tk.DISABLED); log_area.tag_config(color, foreground=color)
def update_displays():
    current_pos_var.set(f"{current_position:.3f} mm")
    for i, p_val in enumerate(saved_positions.values()): saved_pos_vars[i].set(f"{p_val:.3f} mm")
    history_area.config(state=tk.NORMAL); history_area.delete('1.0', tk.END)
    for entry in reversed(position_history): history_area.insert('1.0', entry + "\n")
    history_area.config(state=tk.DISABLED)

# (GUIレイアウト部分は変更なしのため省略します)
# --- GUIレイアウト ---
window = tk.Tk(); window.title("高機能ステッピングモーターコントローラー"); window.protocol("WM_DELETE_WINDOW", on_closing); window.bind("<KeyPress>", handle_key_press); window.geometry("800x750")
default_font = font.nametofont("TkDefaultFont"); default_font.configure(size=10); large_font = default_font.copy(); large_font.configure(size=22, weight="bold")
main_notebook = ttk.Notebook(window); main_notebook.pack(pady=5, padx=10, fill="both", expand=True)
op_frame = ttk.Frame(main_notebook); main_notebook.add(op_frame, text='操作パネル')
settings_frame = ttk.Frame(main_notebook, padding="20"); main_notebook.add(settings_frame, text='設定')
control_panel = ttk.Frame(op_frame); control_panel.pack(fill="x", padx=10, pady=(10,0))
pos_frame = ttk.LabelFrame(control_panel, text="位置制御", padding="10"); pos_frame.pack(side="left", fill="both", expand=True, padx=(0,5))
manual_frame = ttk.LabelFrame(control_panel, text="手動操作", padding="10"); manual_frame.pack(side="left", fill="both", expand=True, padx=(5,0))
ttk.Label(pos_frame, text="現在位置:").pack(anchor="w")
pos_display_frame = tk.Frame(pos_frame, bg='#333'); pos_display_frame.pack(fill="x", pady=2)
current_pos_var = tk.StringVar(value="0.000 mm")
ttk.Label(pos_display_frame, textvariable=current_pos_var, font=large_font, foreground="#00FF00", background="#333", anchor="center").pack(pady=5)
abs_move_frame = ttk.Frame(pos_frame); abs_move_frame.pack(fill="x", pady=(8,0))
goto_pos_entry = ttk.Entry(abs_move_frame); goto_pos_entry.pack(side="left", fill="x", expand=True)
goto_btn = ttk.Button(abs_move_frame, text="目標位置へ移動", command=goto_absolute_position); goto_btn.pack(side="left", padx=(5,0))
origin_btn = ttk.Button(pos_frame, text="原点設定", command=set_origin); origin_btn.pack(fill="x", pady=(5,0))
jog_frame = ttk.Frame(manual_frame); jog_frame.pack(fill="x")
jog_step_var = tk.DoubleVar(value=1.0); ttk.Label(jog_frame, text="ジョグ:").pack(side="left")
r1 = ttk.Radiobutton(jog_frame, text="1mm", variable=jog_step_var, value=1.0); r1.pack(side="left")
r2 = ttk.Radiobutton(jog_frame, text="0.1mm", variable=jog_step_var, value=0.1); r2.pack(side="left")
r3 = ttk.Radiobutton(jog_frame, text="0.05mm", variable=jog_step_var, value=0.05); r3.pack(side="left")

btn_jog_frame = ttk.Frame(manual_frame); btn_jog_frame.pack(fill="x", pady=2)
jog_back_btn = ttk.Button(btn_jog_frame, text="◀", command=jog_backward); jog_back_btn.pack(side="left", fill="x", expand=True)
jog_fwd_btn = ttk.Button(btn_jog_frame, text="▶", command=jog_forward); jog_fwd_btn.pack(side="left", fill="x", expand=True)
ttk.Separator(manual_frame, orient='horizontal').pack(fill='x', pady=8)
saved_pos_vars = [tk.StringVar() for _ in range(3)]; pos_btns = []
for i, p_name in enumerate(saved_positions.keys()):
    p_frame = ttk.Frame(manual_frame); p_frame.pack(fill="x", pady=1)
    ttk.Label(p_frame, text=f"{p_name}:", width=4).pack(side="left"); ttk.Label(p_frame, textvariable=saved_pos_vars[i], width=10).pack(side="left", padx=2)
    b1 = ttk.Button(p_frame, text="登録", width=4, command=lambda name=p_name: save_position(name)); b1.pack(side="left", padx=2)
    b2 = ttk.Button(p_frame, text="移動", width=4, command=lambda name=p_name: goto_position(name)); b2.pack(side="left", padx=2)
    b3 = ttk.Button(p_frame, text="リセット", width=6, command=lambda name=p_name: reset_position(name)); b3.pack(side="left", padx=2)
    pos_btns.extend([b1,b2,b3])
status_var = tk.StringVar(); status_label = ttk.Label(manual_frame, textvariable=status_var, padding=5, anchor="center", font=("Helvetica", 10, "bold")); status_label.pack(fill="x", pady=(5,0))
stop_button = tk.Button(manual_frame, text="緊急停止", bg="red", fg="white", font=font.nametofont("TkDefaultFont"), command=emergency_stop); stop_button.pack(fill="x", pady=(8,0))
lower_notebook = ttk.Notebook(op_frame); lower_notebook.pack(pady=10, padx=10, fill="both", expand=True)
seq_frame = ttk.Frame(lower_notebook, padding="10"); lower_notebook.add(seq_frame, text='シーケンス')
history_frame = ttk.Frame(lower_notebook, padding="10"); lower_notebook.add(history_frame, text='位置履歴')
ttk.Label(seq_frame, text="ステップ").grid(row=0, column=0, padx=5, pady=2); ttk.Label(seq_frame, text="移動距離 (mm)").grid(row=0, column=1, padx=5, pady=2)
ttk.Label(seq_frame, text="速度 (mm/s)").grid(row=0, column=2, padx=5, pady=2); ttk.Label(seq_frame, text="待機 (s)").grid(row=0, column=3, padx=5, pady=2)
seq_entries = []; seq_widgets = []
for i in range(5):
    vars_dict = {'dist_var': tk.StringVar(), 'speed_var': tk.StringVar(), 'wait_var': tk.StringVar()}
    ttk.Label(seq_frame, text=f"{i+1}").grid(row=i+1, column=0)
    dist_e = ttk.Entry(seq_frame, width=15, textvariable=vars_dict['dist_var']); dist_e.grid(row=i+1, column=1, padx=2, pady=2)
    speed_e = ttk.Entry(seq_frame, width=15, textvariable=vars_dict['speed_var']); speed_e.grid(row=i+1, column=2, padx=2, pady=2)
    wait_e = ttk.Entry(seq_frame, width=15, textvariable=vars_dict['wait_var']); wait_e.grid(row=i+1, column=3, padx=2, pady=2)
    seq_entries.append(vars_dict); seq_widgets.extend([dist_e, speed_e, wait_e])
seq_btn_frame = ttk.Frame(seq_frame); seq_btn_frame.grid(row=6, column=1, columnspan=3, pady=10, sticky="e")
seq_run_btn = ttk.Button(seq_btn_frame, text="シーケンス実行", command=run_sequence); seq_run_btn.pack(side="left", padx=5)
seq_clear_btn = ttk.Button(seq_btn_frame, text="クリア", command=clear_sequence); seq_clear_btn.pack(side="left")
history_area = scrolledtext.ScrolledText(history_frame, height=10, state=tk.DISABLED, wrap=tk.NONE); history_area.pack(fill="both", expand=True)
log_frame = ttk.LabelFrame(op_frame, text="ログ", padding="10"); log_frame.pack(pady=5, padx=10, fill="x")
log_area = scrolledtext.ScrolledText(log_frame, height=8, state=tk.DISABLED); log_area.pack(fill="both", expand=True)
pad_options = {'padx': 10, 'pady': 8, 'sticky':'w'}; settings_container = ttk.Frame(settings_frame); settings_container.pack(anchor='center', pady=20)
jog_speed_var = tk.StringVar(); goto_speed_var = tk.StringVar(); seq_speed_var = tk.StringVar()
ttk.Label(settings_container, text="ジョグ移動の速度 (mm/s):").grid(row=0, column=0, **pad_options)
ttk.Entry(settings_container, textvariable=jog_speed_var, width=15).grid(row=0, column=1, **pad_options)
ttk.Label(settings_container, text="絶対位置移動の速度 (mm/s):").grid(row=1, column=0, **pad_options)
ttk.Entry(settings_container, textvariable=goto_speed_var, width=15).grid(row=1, column=1, **pad_options)
ttk.Label(settings_container, text="シーケンスのデフォルト速度 (mm/s):").grid(row=2, column=0, **pad_options)
ttk.Entry(settings_container, textvariable=seq_speed_var, width=15).grid(row=2, column=1, **pad_options)
settings_msg_var = tk.StringVar()
def save_settings_and_show_msg():
    try:
        app_settings["jog_speed"] = float(jog_speed_var.get()); app_settings["goto_speed"] = float(goto_speed_var.get()); app_settings["default_seq_speed"] = float(seq_speed_var.get())
        save_data(); settings_msg_var.set("設定を保存しました。"); log_message("設定を更新しました。"); window.after(2000, lambda: settings_msg_var.set(""))
    except ValueError: settings_msg_var.set("エラー: 数値を入力してください")
ttk.Button(settings_container, text="設定を保存", command=save_settings_and_show_msg).grid(row=3, column=0, columnspan=2, pady=15)
ttk.Label(settings_container, textvariable=settings_msg_var, foreground="blue").grid(row=4, column=0, columnspan=2)
widgets_to_disable.extend([goto_pos_entry, goto_btn, origin_btn, r1, r2, r3, jog_back_btn, jog_fwd_btn, seq_run_btn, seq_clear_btn])
widgets_to_disable.extend(pos_btns); widgets_to_disable.extend(seq_widgets)

# --- メイン処理 ---
load_data(); update_displays(); connect_serial()
window.after(100, read_from_arduino) # Arduinoからの読取ループを開始
window.mainloop()

設定を保存しました。
